<a href="https://colab.research.google.com/github/khusanovich/modul_3.1/blob/main/3_1_2_SQL_2_qism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd 
import sqlite3 as sql

## Yangi ma'limotlar omborini yaratamiz

In [4]:
db = sql.connect("newdb.db")
cursor = db.cursor()

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/Electricity_Production_By_Source.csv")
df.head()

,Entity,Code,Year,Electricity from coal (TWh),Electricity from gas (TWh),Electricity from hydro (TWh),Electricity from other renewables (TWh),Electricity from solar (TWh),Electricity from oil (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh)
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,0.309,0.630,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,0.329,0.565,0.0,0.0,0.0,0.0,0.0


DFni omborga yuklashdan avval, kelin ustunlar nomini qisqartiraylik. Bunday qilishimizga sabab, keyinchalik SQL komandalar yozganda uzun nomlarni yozish o'ziga yarasha noqulayliklar keltirib chiqaradi.

In [6]:
cols = ['country','code','year','coal','gas','hydro','renewable_other','solar','oil','wind','nuclear']
df.columns= cols
df.head()

,country,code,year,coal,gas,hydro,renewable_other,solar,oil,wind,nuclear
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,0.309,0.630,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,0.329,0.565,0.0,0.0,0.0,0.0,0.0


`.to_sql()` metodi yordamida jadvalni to'g'ridan to'g'ri omborga ko'chirishimiz mumkin. 

Jadvalga `jadval` deb yangi nom beramiz.

In [7]:
df.to_sql('jadval', db)

In [9]:
df2 = pd.read_sql("SELECT * FROM jadval", db, index_col="index")

In [10]:
df2.head()

,country,code,year,coal,gas,hydro,renewable_other,solar,oil,wind,nuclear
index,,,,,,,,,,,
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,0.309,0.630,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,0.329,0.565,0.0,0.0,0.0,0.0,0.0


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6241 entries, 0 to 6240
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country          6241 non-null   object 
 1   code             5474 non-null   object 
 2   year             6241 non-null   int64  
 3   coal             5221 non-null   float64
 4   gas              5221 non-null   float64
 5   hydro            6241 non-null   float64
 6   renewable_other  6206 non-null   float64
 7   solar            6241 non-null   float64
 8   oil              5221 non-null   float64
 9   wind             6241 non-null   float64
 10  nuclear          6241 non-null   float64
dtypes: float64(8), int64(1), object(2)
memory usage: 585.1+ KB


# Omborda bo'sh jadval yaratish

**Jadval yaratish qadamlari:**
- `CREATE TABLE` - jadval yaratish buyrug'i
- `IF NOT EXISTS` - jadval mavjud bo'lmasa
- `worldpop` - jadval nomi
- Qavs ichida jadval ustunlari, ustundagi ma'lumot turi va qo'shimcha parametrlar vergul bilan ajratib yoziladi.

**Quyidagi misolda jadvalimiz 5 ta ustundan iborat:**
- `rank` - davlat reytingi (zichlik bo'yicha), `INT` ya'ni butun son.
- `country` - davlat nomi, `TEXT` - matn, `PRIMARY KEY` - birlamchi qiymat, qatorga shu ustun qiymati bilan murojaat qilinadi (takrorlanman bo'lishi kerak)
- `density` - zichlik, `DOUBLE` - o'nlik son
- `population` - aholi soni, `DOUBLE` - o'nlik son, `NOT NULL` - bo'sh bo'lmasligi (qiymat berilishi shart)
- `area` - maydoni, `INT` - butun son, `NOT NULL` - qiymat berilishi shart.

In [12]:
query = """
CREATE TABLE IF NOT EXISTS worldpop(
rank INT,
country TEXT PRIMARY KEY,
density DOUBLE, 
population DOUBLE NOT NULL,
area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()


In [13]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('jadval',), ('worldpop',)]


In [14]:
cursor.execute("PRAGMA table_info(worldpop);")
cursor.fetchall()

[(0, 'rank', 'INT', 0, None, 0),
 (1, 'country', 'TEXT', 0, None, 1),
 (2, 'density', 'DOUBLE', 0, None, 0),
 (3, 'population', 'DOUBLE', 1, None, 0),
 (4, 'area', 'INT', 1, None, 0)]

In [17]:
# jadvalni o'qiymiz
df3 = pd.read_sql("SELECT * FROM worldpop", db)
df3

,rank,country,density,population,area


## Jadvalga yangi ma'lumot qo'shish
SQL jadvlaga ushbu manzilda joylashgan ma'lumotlarni kiritamiz: https://github.com/anvarnarz/praktikum_datasets/blob/main/world_population.csv

![Imgur](https://i.imgur.com/lilTCTm.png)

Keling SQL jadvalga yuqoridan `Macau` davlatiga oid ma'lumotlarni kiritamiz.

Agar barcha ustunlar uchun qiymat berayotgan bo'lsak to'g'ridan-to'g'ri qiymatlarni yozamiz. Bunda qiymatlar tartibi jadval ustunlari ketma-ketligi bilan mos tushishiga ahamiyat bering:


In [19]:
df3

,rank,country,density,population,area


In [20]:
query="INSERT INTO worldpop VALUES (1, 'Macau', 21946.4667, 658.3940, 30)"
cursor.execute(query)
db.commit()

In [24]:
# qiymat qo'shilganini tekshirib ko'ramiz
query= "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

[(1, 'Macau', 21946.4667, 658.394, 30)]

In [25]:
pd.read_sql(query, db)

,rank,country,density,population,area
0,1,Macau,21946.4667,658.394,30


Agar faqatgina ma'lum ustunlarga qiymat qo'shilsa, avval ustun nomlari yoziladi. Keling jadvlaga O'zbekiston haqida quyidagi ma'lumotlarni qo'shamiz:
- `country` = `Uzbekistan`
- `population` = `33935.7630`
- `area` = `447400`

In [26]:
query="""
INSERT INTO worldpop (country, population, area)
VALUES ('Uzbekistan', 33935.7630, 447400)
"""
cursor.execute(query)
db.commit()

In [27]:
# tekshiramiz
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

[(1, 'Macau', 21946.4667, 658.394, 30),
 (None, 'Uzbekistan', None, 33935.763, 447400)]

In [29]:
pd.read_sql(query, db)

,rank,country,density,population,area
0,1.0,Macau,21946.4667,658.394,30
1,NaN,Uzbekistan,NaN,33935.763,447400


Ko'rib turganingizdek mavjud bo'lmagan qiymatlar o'rniga `None` qo'yb ketildi.


## Ma'lumotlarni yangilash/qo'shish
`Uzbekistan` qatoriga `rank` va `density` qiymatlarini qo'shamiz.

In [30]:
query = """
UPDATE worldpop
SET rank=133, density=75.8511
WHERE country = "Uzbekistan"
"""
cursor.execute(query)
db.commit()

In [32]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

[(1, 'Macau', 21946.4667, 658.394, 30),
 (133, 'Uzbekistan', 75.8511, 33935.763, 447400)]

## Jadvaldan ma'lumot o'chirish

In [33]:
query = """
DELETE FROM worldpop
WHERE country="Macau"
"""
cursor.execute(query)
cursor.fetchall()

[]

In [34]:
# tekshiramiz
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall()

[(133, 'Uzbekistan', 75.8511, 33935.763, 447400)]

## Jadvalni butunlay o'chirib tashlash

In [48]:
# cursor.execute("DROP TABLE worldpop")
# db.commit()

In [44]:
print(cursor.fetchall()) 

[]


In [46]:
cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('table', 'jadval', 'jadval', 2, 'CREATE TABLE "jadval" (\n"index" INTEGER,\n  "country" TEXT,\n  "code" TEXT,\n  "year" INTEGER,\n  "coal" REAL,\n  "gas" REAL,\n  "hydro" REAL,\n  "renewable_other" REAL,\n  "solar" REAL,\n  "oil" REAL,\n  "wind" REAL,\n  "nuclear" REAL\n)')]


# SQL buyruqlarni f-string yordamida yozish
Biz yuqorida SQL buyruqlarni oddiy matn ko'rinishida yozayapamiz, demak Pythondagi f-string yordamida matnlarga o'zgaruvchilarning qiymatini ham qo'shish mumkin.

Keling boshlanishiga quyidagi DF ni yaratib olamiz, so'ngra DF qatorlarini yangi SQL jadvalga f-string yordamida yozamiz.

In [49]:
df = pd.read_csv("https://github.com/anvarnarz/praktikum_datasets/raw/main/world_population.csv", index_col='rank')
df.head()

,country,density,densityMi,pop2021,area
rank,,,,,
1,Macau,21946.4667,56841.3487,658.394,30
2,Monaco,19755.5000,51166.7450,39.511,2
3,Singapore,8305.1915,21510.4461,5896.686,710
4,Hong Kong,6841.3134,17719.0017,7552.810,1104
5,Gibraltar,5616.3333,14546.3033,33.698,6


In [50]:
# yangi sql jadval yaratamiz
query = """
CREATE TABLE IF NOT EXISTS worldpop(
rank INT,
country TEXT PRIMARY KEY,
density DOUBLE,
population DOUBLE NOT NULL,
area INT NOT NULL
)
"""
cursor.execute(query)
db.commit()

In [55]:
# jadvaldan Uzbekistan uchun ma'lumotlarni ajratib olamiz
uz = df[df.country == "Uzbekistan"]
print(uz)

         country  density  densityMi    pop2021    area
rank                                                   
133   Uzbekistan  75.8511   196.4542  33935.763  447400


In [58]:
rank = uz.index.values[0]
rank

133

In [60]:
rank = uz.index.values[0]
country = uz.at[rank, 'country']
density = uz.at[rank, 'density']
population = uz.at[rank, 'pop2021']
area = uz.at[rank, 'area']
print(f"{rank}, {country}, {density}, {population}, {area}")

133, Uzbekistan, 75.8511, 33935.763, 447400


In [61]:
query = f"""
INSERT INTO worldpop VALUES ({rank}, '{country}', {density}, {population}, {area}) 
"""
cursor.execute(query)
db.commit()

In [62]:
query = "SELECT * FROM worldpop"
cursor.execute(query)
cursor.fetchall() 

[(133, 'Uzbekistan', 75.8511, 33935.763, 447400)]

In [63]:
db.close()